# Training ML Model with Custom Script Container

**SageMaker Studio Kernel**: Data Science

In this exercise you will do:
 - Run a Preprocessing Job using Amazon SageMaker Processing Job
 - Run a Tensorflow Training Job using Amazon SageMaker Training Job
 - Register a new version of the trained model in the Amazon SageMaker Model Registry

***

## Part 1/2 - Setup
Here we'll import some libraries and define some variables. You can also take a look on the scripts that were previously created for preparing the data and training our model.

In [ ]:
import boto3
from datetime import datetime
import logging
import json
import os
import pandas as pd
import sagemaker
from sagemaker.estimator import Estimator, Framework
from sagemaker import get_execution_role, image_uris
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.model import Model
from sagemaker.tensorflow import TensorFlowModel
import time
import traceback
import yaml

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

In [ ]:
sagemaker_client = boto3.client("sagemaker")
s3_client = boto3.client("s3")

***

### Global configurations

Configuration variables used for Processing, Training, and registration

In [ ]:
region = boto3.session.Session().region_name
role_name = "mlops-sagemaker-execution-role"
role = "arn:aws:iam::{}:role/{}".format(boto3.client('sts').get_caller_identity().get('Account'), role_name)

kms_account_id = boto3.client('sts').get_caller_identity().get('Account')

kms_alias = "ml-kms"

bucket_name = ""

In [ ]:
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client("sagemaker")
runtime_client = boto_session.client("sagemaker-runtime")

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=runtime_client,
    default_bucket=bucket_name
)

In [ ]:
kms_key = "arn:aws:kms:{}:{}:alias/{}".format(region, kms_account_id, kms_alias)

***

## Part 2/2: Run the end to end ML workflow

### Step 1/3: Create the Processing Job

#### Define input variables

In [ ]:
processing_entrypoint = "./../algorithms/processing/src/processing.py"
processing_framework_version = "0.23-1"
processing_instance_count = 1
processing_instance_type = "ml.t3.large"
processing_input_files_path = "data/input"
processing_output_files_path = "data/output"

#### Get the dataset and upload it to an S3 bucket

In [ ]:
# Download the 
# clean the buckets first
s3_client.delete_object(Bucket=bucket_name, Key=processing_input_files_path)

input_data = sagemaker_session.upload_data('./../data/TheSocialDilemma.csv', key_prefix=processing_input_files_path)

LOGGER.info(input_data)

#### Create Processor

Lets create a SKLearn Processor

In [ ]:
!pygmentize ./../algorithms/processing/src/processing.py

In [ ]:
processor = SKLearnProcessor(
    framework_version=processing_framework_version,
    role=role,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
    output_kms_key=kms_key,
    sagemaker_session=sagemaker_session
)

In [ ]:
processor.run(
    code=processing_entrypoint,
    outputs=[
        ProcessingOutput(
            output_name="output", 
            source="/opt/ml/processing/output", 
            destination="s3://{}/{}".format(bucket_name, processing_output_files_path))
    ],
    arguments = [
        "--input-data",
        "s3://{}/{}".format(bucket_name, processing_input_files_path)
    ],
    wait=True
)

***

### Step 2/3: Create the Trining Job

#### Define input variables

#### Compress source code for installing additional python modules

In [ ]:
!pygmentize ./../algorithms/training/src/train.py

In [ ]:
! ./../algorithms/buildspec.sh training $bucket_name

#### Create Estimator

Lets start a training job using a Framework Estimator. Framework estimator allows you to use a custom container created for installing additional dependencies. For providing custom scripts to our container, we have to define the arguments `sagemaker_program` and `sagemaker_submit_directory` as hyperparameters

### Custom Dockerfile

The Dockerfile defined is creating starting from the public tensorflow 2.4.1 image, and by the usage of [sagemaker-training-toolkit](https://github.com/aws/sagemaker-training-toolkit) we are making our container compatibile for Amazon SageMaker

In [ ]:
!pygmentize ./../algorithms/training/Dockerfile

***

In [ ]:
training_artifact_path = "artifact/training"
training_artifact_name = "sourcedir.tar.gz"
training_output_files_path = "models"
training_framework_version = "2.4"
training_python_version = "py37"
training_instance_count = 1
training_instance_type = "ml.p2.xlarge"
training_hyperparameters = {
    "sagemaker_program": "train.py",
    "sagemaker_submit_directory": "s3://{}/{}/{}".format(bucket_name,
                                      training_artifact_path,
                                      training_artifact_name
                                      ),
    "epochs": 5,
    "learning_rage": 3e-5,
    "batch_size": 100,
    "input_file": "processed_data.csv"
}

In [ ]:
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

In [ ]:
estimator = Estimator(
    image_uri="691148928602.dkr.ecr.eu-west-1.amazonaws.com/bert-training-toolkit:latest",
    output_path="s3://{}/{}".format(bucket_name,
                                    training_output_files_path),
    hyperparameters=json_encode_hyperparameters(training_hyperparameters),
    enable_sagemaker_metrics=True,
    metric_definitions=[
        {
            'Name': 'Test accuracy',
            'Regex': 'Test accuracy:.* ([0-9\\.]+)'
        }
    ],
    role=role,
    instance_count=training_instance_count,
    instance_type="local",
    output_kms_key=kms_key,
    disable_profiler=True
)

In [ ]:
estimator.fit(
    inputs={
        "train": "s3://{}/{}".format(
            bucket_name,
            processing_output_files_path
        ),
    },
    logs="Rules"
)

***

### Step 3/3: Register Model in the Model Registry

#### Input Parameters

In [ ]:
inference_instance_type = "ml.m5.xlarge"

model_package_group_name = "ml-end-to-end-group"
model_approval_status = "PendingManualApproval"

In [ ]:
estimator.register(
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type]
)

We have just seen how to process, train, and version ML models by using Amazon SageMaker Jobs. Now we are ready to execute our end to end workflow using an Amazon SageMaker Pipeline

 > [SageMaker-Pipeline](./03-SageMaker-Pipeline-Training.ipynb)